### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

['Selam ',
 'MSG 2 the Messenger ',
 'Avengers: Age of Ultron Parody ',
 'Let There Be Light ',
 'Five Minutes ']

In [97]:
movie_ratings = reviews.groupby('movie_id')['rating']
avg_ratings = movie_ratings.mean()
num_ratings = movie_ratings.count()
last_rating = reviews.groupby('movie_id')['date'].max()
ratings_count_df = pd.DataFrame({'avg_rating':avg_ratings,'num_ratings':num_ratings,'last_rating':last_rating})
ratings_count_df = ratings_count_df[ratings_count_df['num_ratings']>4]
ratings_count_df.sort_values(by=['avg_rating','num_ratings','last_rating'], ascending=False, inplace=True)

In [99]:
movies_recs = movies.set_index('movie_id').join(ratings_count_df)

In [100]:
def create_ratings(movies, reviews):
    movie_ratings = reviews.groupby('movie_id')['rating']
    avg_ratings = movie_ratings.mean()
    num_ratings = movie_ratings.count()
    
    last_rating = reviews.groupby('movie_id')['date'].max()
    ratings_count_df = pd.DataFrame({'avg_rating':avg_ratings,'num_ratings':num_ratings,'last_rating':last_rating})
    
    movies_recs = movies.set_index('movie_id')[['movie']].join(ratings_count_df)
    
    ranked_movies = movies_recs.sort_values(by=['avg_rating','num_ratings','last_rating'], ascending=False)
    ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
    
    return ranked_movies 

def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies # a list of the n_top movies as recommended

In [101]:
ranked_movies = create_ratings(movies, reviews)

In [106]:
popular_recommendations(10, 20, ranked_movies)

['MSG 2 the Messenger ',
 'Avengers: Age of Ultron Parody ',
 'Five Minutes ',
 'Selam ',
 'Let There Be Light ',
 "Quiet Riot: Well Now You're Here, There's No Way Back ",
 'Crawl Bitch Crawl ',
 'Chasing Happiness ',
 'Make Like a Dog ',
 'Pandorica ',
 'Third Contact ',
 'Romeo Juliet ',
 'Be Somebody ',
 'Birlesen Gonuller ',
 'Kitbull ',
 'Agnelli ',
 'Sátántangó ',
 'Foster ',
 'CM101MMXI Fundamentals ',
 'Crystal Lake Memories: The Complete History of Friday the 13th ']

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [107]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20, ranked_movies) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5, ranked_movies)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(1, 100, ranked_movies)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(1, 35, ranked_movies)# Your solution list here



In [108]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [116]:
ranked_movies['History']

movie_id
4921860     0
5262972     0
6662050     0
2737018     0
5804314     0
           ..
10367276    0
3108604     0
3187378     0
4458206     0
7923374     0
Name: History, Length: 11524, dtype: int64

In [136]:
def popular_recommendations_filtered(user_id, n_top, ranked_movies, years_list, genres_list):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    ranked_movies = ranked_movies[ranked_movies['year'].isin(years_list)]
    ranked_movies = ranked_movies[ranked_movies[genres_list].sum(axis=1) > 0 ]    
    
    top_movies = list(ranked_movies['movie'][:n_top])
    return top_movies    

In [137]:
popular_recommendations_filtered(1, 20, ranked_movies, ['2015', '2016', '2017', '2018'], ['History','Short'])

['Avengers: Age of Ultron Parody ',
 'Five Minutes ',
 'Make Like a Dog ',
 "Hillary's America: The Secret History of the Democratic Party ",
 'I Believe in Miracles ',
 'Inner Workings ',
 'O.J.: Made in America ',
 'Ayla: The Daughter of War ',
 'Hacksaw Ridge ',
 'They Shall Not Grow Old ',
 'Namhansanseong ',
 'The Farthest ',
 'Piper ',
 'Kono sekai no katasumi ni ',
 'Sado ',
 'Silicon Cowboys ',
 'World of Tomorrow ',
 '13th ',
 'Ethel &amp; Ernest ',
 'Paul, Apostle of Christ ']

In [130]:
ranked_movies[['History','Short']].sum(axis=1) > 0

movie_id
4921860     False
5262972      True
6662050      True
2737018     False
5804314     False
            ...  
10367276     True
3108604     False
3187378     False
4458206     False
7923374     False
Length: 11524, dtype: bool